In [3]:
#Importando as libs usadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#Lendo o dataset
input_df = pd.read_csv('../input/circuito-yy/Input.csv', sep=';')
valid_df = pd.read_csv('../input/v2-valid/Validao v2.csv', encoding='unicode_escape', sep=';')

FileNotFoundError: [Errno 2] No such file or directory: '../input/circuito-yy/Input.csv'

In [ ]:
#Verificando os dados input e como estão dispostos
input_df

In [ ]:
#Removendo os valores NaN's pelo fato de que 
input_df.dropna(inplace = True)

In [ ]:
#Alterando o título DATA para datetime e alterando o título tensao_f1_f2 para tensao_linha_f1_f2 que é a tensão
#entre fases
input_df.rename({'DATA':'datetime', 'tensao_f1_f2':'tensao_linha_f1_f2'}, axis=1, inplace=True)

In [ ]:
#Organizando os dados por data e hora
input_org = input_df.sort_values(by="datetime")

In [ ]:
#Removendo a coluna de tensão de linha, pois não seria tão interessante pra gente 
input_org.drop(['tensao_linha_f1_f2'], axis = 1, inplace = True)

In [ ]:
#Visualizando os dados já organizados
input_org

In [ ]:
#Levando em consideração que as tensões tem as mesmas amplitudes e frequências, temos
input_org['tensao_f3'] = input_org['tensao_f1']

In [ ]:
#Visualizando os dados já organizados
input_org

In [ ]:
#Adicionando as infos de ângulo de fases
input_org['angulo_tensao_f1'] = 0
input_org['angulo_tensao_f2'] = 119.7
input_org['angulo_tensao_f3'] = -120.1
input_org['fp'] = 0.86

In [ ]:
#Visualizando os dados já organizados
input_org

In [ ]:
#Novo dataframe
report_df = input_org.filter(items=['datetime'])

In [ ]:
#Antes de começarmos os cálculos das potências precisamos saber a informação de ângulo de fase da corrente
#Considerando que as cargas são indutivas temos o fp atrasado
#Como FP = Cos(theta) e theta = (theta v) - (theta i), ficamos com:
theta = np.degrees(np.arccos(input_org['fp']))
#Logo, os angulos de fase das correntes
theta_i_f1 = input_org['angulo_tensao_f1'] - theta
theta_i_f2 = input_org['angulo_tensao_f2'] - theta
theta_i_f3 = input_org['angulo_tensao_f3'] - theta
#print(theta)
#print(theta_i_f1)
#print(theta_i_f2)
#print(theta_i_f3)

In [ ]:
#Calculando o terceiro termo da equação P = V(rms)*I(rms)*cos(theta(v)-theta(i))
cos_theta_f1 = np.cos(np.radians(input_org['angulo_tensao_f1'] - theta_i_f1))
cos_theta_f2 = np.cos(np.radians(input_org['angulo_tensao_f2'] - theta_i_f2))
cos_theta_f3 = np.cos(np.radians(input_org['angulo_tensao_f3'] - theta_i_f3))

#print(cos_theta_f1)
#print(cos_theta_f2)
#print(cos_theta_f3)

In [ ]:
# Considerando a Potência ativa para o regime permanente senoidal, temos P = V(rms)*I(rms)*cos(theta(v)-theta(i))
potencia_ativa_f1 = input_org['corrente_f1']*input_org['tensao_f1']*cos_theta_f1
potencia_ativa_f2 = input_org['corrente_f2']*input_org['tensao_f2']*cos_theta_f2
potencia_ativa_f3 = input_org['corrente_f2']*input_org['tensao_f2']*cos_theta_f3

#print(potencia_ativa_f1)
#print(potencia_ativa_f2)
#print(potencia_ativa_f3)

In [ ]:
#Calculando os consumos para 1 min, que é o tempo de cada datetime
energia_ativa_f1 = potencia_ativa_f1/1000*0.017
energia_ativa_f2 = potencia_ativa_f2/1000*0.017
energia_ativa_f3 = potencia_ativa_f3/1000*0.017

#energia_ativa_total
energia_ativa_total = energia_ativa_f1 + energia_ativa_f2 + energia_ativa_f3

#print(energia_ativa_f1)
#print(energia_ativa_f2)
#print(energia_ativa_f3)
#print(energia_ativa_total)

In [ ]:
#Criando títulos no dataframe
report_df['fator_potencia'] = input_org['fp']
report_df['energia_ativa_f1_(KWh)'] = energia_ativa_f1
report_df['energia_ativa_f2_(KWh)'] = energia_ativa_f2
report_df['energia_ativa_f3_(KWh)'] = energia_ativa_f3
report_df['energia_ativa_total_(KWh)'] = energia_ativa_total

In [ ]:
#Calculando o terceiro termo da equação Q = V(rms)*I(rms)*sin(theta(v)-theta(i))
sin_theta_f1 = np.sin(np.radians(input_org['angulo_tensao_f1']) - theta_i_f1)
sin_theta_f2 = np.sin(np.radians(input_org['angulo_tensao_f2']) - theta_i_f2)
sin_theta_f3 = np.sin(np.radians(input_org['angulo_tensao_f3']) - theta_i_f3)

In [ ]:
#Calculando a potência reativa
potencia_reativa_f1 = input_org['corrente_f1']*input_org['tensao_f1']*sin_theta_f1
potencia_reativa_f2 = input_org['corrente_f2']*input_org['tensao_f2']*sin_theta_f2
potencia_reativa_f3 = input_org['corrente_f2']*input_org['tensao_f2']*sin_theta_f3

In [ ]:
#Calculando os consumos para 1 min, que é o tempo de cada datetime
energia_reativa_f1 = potencia_reativa_f1/1000*0.017
energia_reativa_f2 = potencia_reativa_f2/1000*0.017
energia_reativa_f3 = potencia_reativa_f3/1000*0.017
energia_reativa_total = energia_reativa_f1 + energia_reativa_f2 + energia_reativa_f3

In [ ]:
#Criando títulos no dataframe
report_df['energia_reativa_f1_(KVArh)'] = energia_reativa_f1
report_df['energia_reativa_f2(KVArh)'] = energia_reativa_f2
report_df['energia_reativa_f3(KVArh)'] = energia_reativa_f3
report_df['energia_reativa_total_(KVArh)'] = energia_reativa_total

In [ ]:
#Calculos de energia aparente
energia_aparente_f1 = ((potencia_ativa_f1/input_org['fp'])/1000)*0.017
energia_aparente_f2 = ((potencia_ativa_f2/input_org['fp'])/1000)*0.017
energia_aparente_f3 = ((potencia_ativa_f3/input_org['fp'])/1000)*0.017
energia_aparente_total = energia_ativa_f1 + energia_aparente_f2 + energia_aparente_f3

In [ ]:
#Criando títulos no dataframe
report_df['energia_aparente_f1_(KVAh)'] = energia_ativa_f1
report_df['energia_aparente_f2(KVAh)'] = energia_aparente_f2
report_df['energia_aparente_f3(KVAh)'] = energia_aparente_f3
report_df['energia_aparente_total_(KVAh)'] = energia_aparente_total

In [ ]:
report_df

In [ ]:
#cuspindo o arquivo em csv
report_df.to_csv('circuito_estrela.csv', index=False)